In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_html('https://fbref.com/en/comps/Big5/defense/players/Big-5-European-Leagues-Stats',flavor='lxml')[0]

# Get the second level values as a list
second_level_values = df.columns.get_level_values(1).tolist()


df.columns = second_level_values

# Drop rows that contain column names as row values and goalkeepers
df = df[~df.apply(lambda row: row.isin(['Rk','GK'])).any(axis=1)]


# clean the age column

df['Age'] = df['Age'].str.split("-").str[0]

# calculate the minutes played

df['Minutes'] = df['90s'].astype(float) * 90

# drop nas which represent no game time

df = df.dropna()


df = df.loc[df['Minutes']>=1260] # 14 games


# Recode position values using label encoder

le = LabelEncoder()

df['Pos'] = le.fit_transform(df['Pos'])

# Recode league values using label encoder
le_comps = LabelEncoder()
df['Comp'] = le_comps.fit_transform(df['Comp'])

# Get the dictionary of label encoder's encoded values to original values
label_encoding_dict = {label: original_value for label, original_value in zip(le.transform(le.classes_), le.classes_)}

# Get the dictionary of label encoder's encoded values to original values
label_encoding_comp_dict = {label: original_value for label, original_value in zip(le_comps.transform(le_comps.classes_), le_comps.classes_)}

In [2]:
label_encoding_dict

{0: 'DF',
 1: 'DF,FW',
 2: 'DF,MF',
 3: 'FW',
 4: 'FW,DF',
 5: 'FW,MF',
 6: 'MF',
 7: 'MF,DF',
 8: 'MF,FW'}

In [3]:
label_encoding_comp_dict

{0: 'de Bundesliga',
 1: 'eng Premier League',
 2: 'es La Liga',
 3: 'fr Ligue 1',
 4: 'it Serie A'}

In [4]:
df = df.drop(['Squad','Nation','Rk','Born','Matches'],axis=1)

In [5]:
df = df.sort_values('Minutes',ascending=False)

In [6]:
duplicate_players = df[df.duplicated('Player')]
duplicate_players

,Player,Pos,Comp,Age,90s,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,...,Tkl%,Lost,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err,Minutes
2348,Rodrigo,5,2,22,17.8,16,11,5,9,2,...,47.1,9,10,0,10,13,29,2,0,1602.0
1093,Nicolás González,5,4,24,15.0,17,10,3,10,4,...,31.3,11,14,0,14,4,21,6,0,1350.0
1908,Álex Moreno,0,1,29,14.8,23,14,15,5,3,...,36.1,23,18,3,15,7,30,37,2,1332.0
1884,Terem Moffi,3,3,23,14.8,12,6,0,6,6,...,16.7,10,3,0,3,2,14,5,0,1332.0
2664,Issiaga Sylla,0,3,28,14.5,28,16,19,8,1,...,48.4,16,19,3,16,32,60,35,0,1305.0


In [7]:
df.loc[df['Player'] == 'Terem Moffi']

,Player,Pos,Comp,Age,90s,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,...,Tkl%,Lost,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err,Minutes
1885,Terem Moffi,3,3,23,15.4,8,5,1,4,3,...,60.0,2,11,3,8,3,11,10,0,1386.0
1884,Terem Moffi,3,3,23,14.8,12,6,0,6,6,...,16.7,10,3,0,3,2,14,5,0,1332.0


In [8]:
df = df.drop_duplicates(['Player'],keep='first')

check for duplicates again

In [9]:
duplicate_players = df[df.duplicated('Player')]
duplicate_players

,Player,Pos,Comp,Age,90s,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,...,Tkl%,Lost,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err,Minutes


In [11]:
df.columns = ['Player','Position','League','Age','90s','Players_tackled','Tackles_won','Tackles_def3rd','Tackles_mid3rd','Tackles_att3rd','Dribblers_tackled','Dribblers_challenged',
              'Dribbers_tackled%','Challenges_lost','Number_of_blocks','Shots_blocked','Passes_blocked','Interceptions','Number_of_Players_tackles_and_Int','Clearances','Errors_leading_to_shot','Minutes']

In [12]:
df.to_csv('clustering_model.csv',index=False)